In [501]:
import pandas as pd
import json
import requests
from matplotlib import pyplot as plt

In [502]:
# STATS THAT I WANT TO RECORD:
# Gametype winrate, accuracy by weapon, map winrate, k/d by gametype, k/d by map, assists?, objective?

In [503]:
player_gamertags = ['ElamAce', 'Sparty TG', 'Wumbology']

player_data = {}
# rank, winrate, games played, favorite gametype, favorite weapon

player_data_gbv = {}
# winrate, objective data?, k/d, assists?, number of games

# player_data_gbv_df = pd.DataFrame()

player_data_maps = {}
# winrate, k/d,  

player_data_weapons = {}
# accuracy per weapon, kills per weapon

gbv = ['Strongholds',
 'Slayer',
 'Breakout',
 'Capture the Flag',
 'Warzone Assault',
 'Warzone',
 'Campaign',
 'Big Team CTF',
 'Big Team Strongholds',
 'Big Team Slayer',
  None,
 'Assault',
 'Grifball',
 'Warzone Firefight',
 'Infection',
 'Campaign Score Attack',
 'Mini-game',
 'Oddball',
 'Big Team Assault']

class Api:
    """  
    This class will call the Halo 5 API and create a dataframe with data for each of
    the provided gamertags
    """
    __api_key = '7022fb201e5a4b35bdf2104472004f9c'
    
    game_base_variants = {}
    
    headers = {
    'Ocp-Apim-Subscription-Key' : __api_key
}
    
    params = {
    'players' : player_gamertags,
}
    
    arena_service_record = 'https://www.haloapi.com/stats/h5/servicerecords/arena'
    
    gbv_url = 'https://www.haloapi.com/metadata/h5/metadata/game-base-variants'
    
    def __init__(self):
        pass
    
    def retrieve_gbv_data(self):
        req = requests.get(self.arena_service_record, params=self.params, headers=self.headers)
        data = req.json()
        for gamertags in range(len(player_gamertags)):
            player_data_gbv[player_gamertags[gamertags]] = pd.DataFrame(data['Results'][gamertags]['Result']['ArenaStats']['ArenaGameBaseVariantStats'])
    
    def retrieve_metadata(self, url):
        self.url = url
        req = requests.get(self.url, headers = self.headers)
        data = req.json()
        self.game_base_variants = pd.DataFrame(data)
        
class DataClean:
    """
    This class will replace the the Id values with metadata values retrieved from the metadata API.
    It will also remove unneeded columns and and construct a dataframe of desired data.
    """
    
    metadata_id = {}
    
    def __init__(self):
        call_api = None
        metadata_replace = None
        metadata_id = None
    
    def gbv_data_clean(self):
        
        self.call_api = Api()
        self.call_api.retrieve_metadata(self.call_api.gbv_url)
        self.metadata_replace = pd.DataFrame(self.call_api.game_base_variants)
        self.metadata_id = pd.Series(self.metadata_replace.name.values,index=self.metadata_replace.id).to_dict()
        
        for i in range(len(player_gamertags)):
            for x in range(len(player_data_gbv[player_gamertags[i]]['GameBaseVariantId'])):
                # this takes the metadata taken from the API and replaces the metadata id with the actual game mode name
                player_data_gbv[player_gamertags[i]]['GameBaseVariantId'][x] = self.metadata_id[player_data_gbv[player_gamertags[i]]['GameBaseVariantId'][x]]
        
        for i in range(len(player_gamertags)):
            # renames the column from GameBaseVariantId to Game Mode
            player_data_gbv[player_gamertags[i]].rename(columns = {
                'GameBaseVariantId' : 'Game Mode'
            }, inplace=True)
        
        for x in range(len(player_gamertags)):
            # adds a column for win percentage based on total games/games won*100
            # adds a column for kill/death ration based on total kills/total deaths
            
            player_data_gbv[player_gamertags[x]]['Win Percentage'] = (player_data_gbv[player_gamertags[x]]['TotalGamesWon'] / player_data_gbv[player_gamertags[x]]['TotalGamesCompleted'])*100
            player_data_gbv[player_gamertags[x]]['K/D Ratio'] = player_data_gbv[player_gamertags[x]]['TotalKills'] / player_data_gbv[player_gamertags[x]]['TotalDeaths']
            player_data_gbv[player_gamertags[x]]['PowerWeaponGrabs/Game'] = player_data_gbv[player_gamertags[x]]['TotalPowerWeaponGrabs'] / player_data_gbv[player_gamertags[x]]['TotalGamesCompleted']
        
        for x in range(len(player_gamertags)):
            player_data_gbv[player_gamertags[x]] = player_data_gbv[player_gamertags[x]][['Game Mode', 'Win Percentage', 'TotalGamesCompleted', 'K/D Ratio', 'TotalKills', 'TotalDeaths', 'TotalAssists', 'PowerWeaponGrabs/Game', 'TotalGamesWon', 'TotalGamesLost']]
            player_data_gbv[player_gamertags[x]].set_index('Game Mode', inplace=True)
                
class CompareStats:
    """
    
    """
    def __init__(self):
        self.call_api = Api()
        self.call_api.retrieve_gbv_data()
        self.replace_metadata = DataClean()
        self.replace_metadata.gbv_data_clean()
        
        self.gbv_dataframe = None
        self.gbv_columns = ['Win Percentage', 'TotalGamesCompleted', 'K/D Ratio', 'TotalKills', 'TotalDeaths', 'TotalAssists', 'PowerWeaponGrabs/Game', 'TotalGamesWon', 'TotalGamesLost']
        
    def player_data(self):
        pass
    
    def gbv_data(self):
        #
        
        self.gbv_dataframe = pd.DataFrame()
        self.gbv_dataframe['Game Mode'] = gbv
        # making a dataframe where the index will be all of the possible gametypes
    
        for i in range(len(self.gbv_columns)):
            for x in range(len(player_gamertags)):
                # make a column named stats: gamertag for each of the gamertags that are being compared
                self.gbv_dataframe[f'{self.gbv_columns[i]}: {player_gamertags[x]}'] = None
        
          
        for i in range(len(self.gbv_dataframe)):
            for x in range(len(player_gamertags)):
                # if the gamertags provided have data for the gametype, then created a column for that player for each of the
                # stats we are including
                if self.gbv_dataframe['Game Mode'][i] in player_data_gbv[player_gamertags[x]].index:
                    for c in range(len(self.gbv_columns)):
                        self.gbv_dataframe[f'{self.gbv_columns[c]}: {player_gamertags[x]}'][i] = player_data_gbv[player_gamertags[x]][f'{self.gbv_columns[c]}'][self.gbv_dataframe['Game Mode'][i]]
        
        self.gbv_dataframe.set_index('Game Mode', inplace=True)
        self.gbv_dataframe.dropna(how='all', inplace=True)
        # drop all rows (gametypes) that were not played at all by any of the given gamertags
        
    def map_data(self):
        pass
    
    def weapon_data(self):
        pass
            

In [504]:
player_data_gbv[player_gamertags[0]]['Win Percentage']['Strongholds']

KeyError: 'ElamAce'

In [505]:
halo5 = CompareStats()
halo5.gbv_data()
# halo5.gbv_dataframe['Game Mode']

C:\Users\hunte\anaconda3\lib\site-packages\ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [506]:
halo5 = CompareStats()
halo5.gbv_data()
halo5.gbv_dataframe

C:\Users\hunte\anaconda3\lib\site-packages\ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Win Percentage: ElamAce,Win Percentage: Sparty TG,Win Percentage: Wumbology,TotalGamesCompleted: ElamAce,TotalGamesCompleted: Sparty TG,TotalGamesCompleted: Wumbology,K/D Ratio: ElamAce,K/D Ratio: Sparty TG,K/D Ratio: Wumbology,TotalKills: ElamAce,...,TotalAssists: Wumbology,PowerWeaponGrabs/Game: ElamAce,PowerWeaponGrabs/Game: Sparty TG,PowerWeaponGrabs/Game: Wumbology,TotalGamesWon: ElamAce,TotalGamesWon: Sparty TG,TotalGamesWon: Wumbology,TotalGamesLost: ElamAce,TotalGamesLost: Sparty TG,TotalGamesLost: Wumbology
Game Mode,,,,,,,,,,,,,,,,,,,,,
Strongholds,76.7234,77.1153,56,2466,3522,25,1.47033,1.64416,1.61257,35157,...,130,0.387672,0.395798,0.88,1892,2716,14,574,805,11
Slayer,69.7364,75.6109,55.6338,3301,4297,142,1.41601,1.81846,1.51065,49188,...,630,0.827325,0.834536,0.485915,2302,3249,79,997,1045,62
Breakout,71.5447,77.836,52,246,573,25,1.5789,2.27542,1.66316,1691,...,37,0.0284553,0.247818,0,176,446,13,70,127,12
Capture the Flag,73.5306,76.6241,54.8387,2501,3602,31,1.37832,1.64686,1.69892,37591,...,160,0.670532,0.644919,0.741935,1839,2760,17,645,799,14
Big Team CTF,100,50,None,3,2,None,2.25,2.85714,None,54,...,None,0.333333,0,None,3,1,None,0,1,None
Big Team Strongholds,75,100,None,8,2,None,2.02128,2.36364,None,190,...,None,0.875,0,None,6,2,None,2,0,None
Big Team Slayer,77.7778,50,100,9,2,1,1.77922,2.42857,11.5,137,...,7,0.777778,0,1,7,1,1,2,1,0
Assault,76.8182,76.6962,50,220,339,2,1.6122,1.93473,1.46667,4203,...,9,0.8,0.728614,0,169,260,1,46,66,1
Grifball,62.5,66.6667,None,8,9,None,1.72034,1.21354,None,203,...,None,0,0,None,5,6,None,3,3,None


In [482]:
player_data_gbv[player_gamertags[0]]

,Win Percentage,TotalGamesCompleted,K/D Ratio,TotalKills,TotalDeaths,TotalAssists,TotalPowerWeaponGrabs,TotalGamesWon,TotalGamesLost
Game Mode,,,,,,,,,
Strongholds,76.723439,2466,1.470327,35157,23911,18318,956,1892,574
Breakout,71.544715,246,1.578898,1691,1071,630,7,176,70
Slayer,69.736444,3301,1.416012,49188,34737,21990,2731,2302,997
Big Team Slayer,77.777778,9,1.779221,137,77,58,7,7,2
Big Team CTF,100.000000,3,2.250000,54,24,25,1,3,0
Oddball,66.359447,217,1.256232,3981,3169,1963,99,144,73
Capture the Flag,73.530588,2501,1.378323,37591,27273,18966,1677,1839,645
Mini-game,100.000000,2,1.727273,19,11,5,0,2,0
Assault,76.818182,220,1.612198,4203,2607,1994,176,169,46


In [483]:
df = pd.DataFrame(data=None, columns=['Game Mode', 'Win Percentage', 'TotalGamesCompleted', 'K/D Ratio', 'TotalKills', 'TotalDeaths', 'TotalAssists', 'TotalPowerWeaponGrabs', 'TotalGamesWon', 'TotalGamesLost'])

In [484]:
df

,Game Mode,Win Percentage,TotalGamesCompleted,K/D Ratio,TotalKills,TotalDeaths,TotalAssists,TotalPowerWeaponGrabs,TotalGamesWon,TotalGamesLost
